# CONTENT BASED FILTERING

## DATA DAN FUNGSI

In [1]:
import pandas as pd
import numpy as np

movies = ["Terminator 2","Interstellar","Ant Man 2","3 Idiots"]
scores = [7,9,8,9]
action = [1,0,1,0]
scifi = [1,1,1,0]
adventure = [0,1,1,0]
comedy = [0,0,1,1]
drama = [0,1,0,1]

df_movies = pd.DataFrame({
    'movie':movies,
    'scores':scores,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})

movies = ["Titanic",'Martian','GOTG Vol 2']
action = [1,0,1]
scifi = [1,1,1]
adventure = [0,1,1]
comedy = [0,0,1]
drama = [0,1,0]

df_movies_recommendation = pd.DataFrame({
    'movie':movies,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})

def user_recommendation(df_movies,df_movies_recommendation):
  df_movies2 = df_movies.copy()
  df_movies2.drop('movie', axis = 1, inplace = True)

  for i in df_movies.columns[2:]:
    df_movies2[i] = df_movies2['scores']*df_movies2[i] 

  df_movies2.drop('scores', axis = 1, inplace = True)
  movie_scoring = df_movies2.sum()/df_movies2.sum().sum()

  for i in df_movies.columns[2:]:
    df_movies_recommendation[i] = df_movies_recommendation[i]*movie_scoring[i]

  df_movies_recommendation['movie rating prediction'] = df_movies_recommendation.sum(axis = 1)
  return df_movies_recommendation[['movie','movie rating prediction']]

In [2]:
df_movies

,movie,scores,Action,Sci-Fi,Adventure,Comedy,Drama
0,Terminator 2,7,1,1,0,0,0
1,Interstellar,9,0,1,1,0,1
2,Ant Man 2,8,1,1,1,1,0
3,3 Idiots,9,0,0,0,1,1


In [3]:
df_movies_recommendation

,movie,Action,Sci-Fi,Adventure,Comedy,Drama
0,Titanic,1,1,0,0,0
1,Martian,0,1,1,0,1
2,GOTG Vol 2,1,1,1,1,0


In [4]:
user_recommendation(df_movies,df_movies_recommendation)

,movie,movie rating prediction
0,Titanic,0.428571
1,Martian,0.648352
2,GOTG Vol 2,0.802198


## DATA & FUNGSI: CONTECT BASED FILTERING MULTIPLE

In [5]:
movies = ["Terminator 2","Interstellar","Ant Man 2","3 Idiots"]
action = [1,0,1,0]
scifi = [1,1,1,0]
adventure = [0,1,1,0]
comedy = [0,0,1,1]
drama = [0,1,0,1]

df_item_features = pd.DataFrame({
    'movie':movies,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})

user = ['user 1','user 2','user 3','user 4']
terminator_2 = [7,8,9,0]
interstellar = [9,0,0,7]
ant_man_2 = [8,6,0,0]
three_idiots = [9,5,10,9]

df_user_items = pd.DataFrame({
    'user':user,
    'Terminator 2':terminator_2,
    'Interstellar':interstellar,
    'Ant Man 2':ant_man_2,
    '3 Idiots':three_idiots
})

movies = ["Titanic","Martian","GOTG Vol 2"]
action = [1,0,1]
scifi = [1,1,1]
adventure = [0,1,1]
comedy = [0,0,1]
drama = [0,1,0]

df_item_features_new = pd.DataFrame({
    'movie':movies,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})


def user_recommendation_multiple(df_user_items,df_item_features,df_item_features_new):
  
  arr_user_items = np.array(df_user_items.drop('user', axis = 1))
  arr_item_features = np.array(df_item_features.drop('movie', axis = 1))

  n_user = arr_user_items.shape[0]
  n_item = arr_user_items.shape[1]
  n_feature = arr_item_features.shape[1]

  arr_user_items_score = np.empty((n_user,n_item))
  arr_user_feature = np.empty((n_user,n_feature))

  for i in range(0,n_user):
    # print(arr_user_items[i,:])
    user_feature = np.matmul(arr_user_items[i,:],arr_item_features)
    # print(user_feature)
    user_feature = user_feature/user_feature.sum()
    arr_user_feature[i,:] = user_feature

  df_user_feature = pd.DataFrame(arr_user_feature)
  df_user_feature.columns = df_item_features.columns[1:]
  df_user_feature.index = user

  for i in range(0, n_user):
    user_item_score = np.matmul(arr_item_features,arr_user_feature[i,:])
    arr_user_items_score[i,:] = user_item_score

  arr_user_items_score_unwatched = np.where(arr_user_items == 0,arr_user_items_score,0)

  df_user_items_score_unwatched = pd.DataFrame(arr_user_items_score_unwatched)
  df_user_items_score_unwatched.columns = df_user_items.columns[1:]
  df_user_items_score_unwatched.index = df_user_items['user']

  arr_item_features_new = np.array(df_item_features_new.drop('movie', axis = 1))

  n_item_new = df_item_features_new.shape[0]

  arr_user_items_score_new = np.empty((n_user,n_item_new))

  for i in range(0, n_user):
    user_item_score = np.matmul(arr_item_features_new,arr_user_feature[i,:])
    arr_user_items_score_new[i,:] = user_item_score

  df_user_items_score_new = pd.DataFrame(arr_user_items_score_new)
  df_user_items_score_new.index = user
  df_user_items_score_new.columns = df_item_features_new['movie']

  return [df_user_items_score_unwatched,df_user_items_score_new]

In [6]:
df_item_features

,movie,Action,Sci-Fi,Adventure,Comedy,Drama
0,Terminator 2,1,1,0,0,0
1,Interstellar,0,1,1,0,1
2,Ant Man 2,1,1,1,1,0
3,3 Idiots,0,0,0,1,1


In [7]:
df_user_items

,user,Terminator 2,Interstellar,Ant Man 2,3 Idiots
0,user 1,7,9,8,9
1,user 2,8,0,6,5
2,user 3,9,0,0,10
3,user 4,0,7,0,9


In [8]:
df_item_features_new

,movie,Action,Sci-Fi,Adventure,Comedy,Drama
0,Titanic,1,1,0,0,0
1,Martian,0,1,1,0,1
2,GOTG Vol 2,1,1,1,1,0


TUJUAN:
1. REKOMENDASI FILM LAMA YG BLM DI TONTON
1. REKOMENDASI FULM BARU UNTUK USERS

In [11]:
rekomendasi_film_lama, rekomendasi_film_baru = user_recommendation_multiple(df_user_items,df_item_features,df_item_features_new)

In [14]:
df_user_items

,user,Terminator 2,Interstellar,Ant Man 2,3 Idiots
0,user 1,7,9,8,9
1,user 2,8,0,6,5
2,user 3,9,0,0,10
3,user 4,0,7,0,9


In [12]:
rekomendasi_film_lama

,Terminator 2,Interstellar,Ant Man 2,3 Idiots
user,,,,
user 1,0.000000,0.0,0.000000,0.0
user 2,0.000000,0.5,0.000000,0.0
user 3,0.000000,0.5,0.736842,0.0
user 4,0.179487,0.0,0.589744,0.0


In [13]:
rekomendasi_film_baru

movie,Titanic,Martian,GOTG Vol 2
user 1,0.428571,0.648352,0.802198
user 2,0.560000,0.500000,0.900000
user 3,0.473684,0.500000,0.736842
user 4,0.179487,0.769231,0.589744


THRESHOLD 0.6

REKOMENDASI:
1. USER 1: GOTG VOL 2, MARTIAN
1. USER 2: GOTG VOL 2
1. USER 3: GOTG VOL 2
1. USER 4: MARTIAN

# COLLABORATIVE FILTERING MODEL BASED

USE U DATA:
- BUILD SVD AND ALS MODEL
- EVALUATE SVD AND ALS MODEL USING VALIDATION
- EVALUATE SVF AND ALS MODEL USING CROSS VALIDATION
- OPTIMIZE SVD USING HYPERPARAMETER TUNING AND COMPARE (BEFORE - AFTER)
- GENERATE THE PREDICTION RESULT

## LIBRARY

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns

from surprise import Reader
from surprise import Dataset

from surprise import SVD
from surprise import BaselineOnly

from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise.model_selection import GridSearchCV

## DATA

In [23]:
columns_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(r'C:\Users\dheof\Desktop\u_data.csv', sep = '\t', names = columns_names)
df

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742
...,...,...,...,...
99998,880,476,3,880175444
99999,716,204,5,879795543
100000,276,1090,1,874795795
100001,13,225,2,882399156


In [24]:
cross_user = df.pivot_table(values='rating', index = 'user_id', columns = 'item_id')
cross_user

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
reader = Reader(rating_scale = (1,5))
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)
data.df

,user_id,item_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3
...,...,...,...
99998,880,476,3
99999,716,204,5
100000,276,1090,1
100001,13,225,2


## MODELING SVD ALS: VALIDATE

In [27]:
trainset, testset = train_test_split(
    data, 
    test_size = 0.25, 
    random_state = 101)

## SVD

In [28]:
algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

In [29]:
accuracy.rmse(predictions)

RMSE: 0.9381


0.9381144568498601

## ALS

In [30]:
bsl_option = {
    'method' : 'als'
    }

algo = BaselineOnly(bsl_options = bsl_option)
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...


In [31]:
accuracy.rmse(predictions)

RMSE: 0.9433


0.943287541263747

## MODELING SVD ALS: CROSS VALIDATE

SVD

In [32]:
algo = SVD()

cv_std = cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9387  0.9278  0.9376  0.9340  0.9422  0.9361  0.0049  
MAE (testset)     0.7409  0.7342  0.7382  0.7370  0.7419  0.7384  0.0028  
Fit time          5.42    5.79    5.24    5.27    5.18    5.38    0.22    
Test time         0.15    0.17    0.20    0.13    0.21    0.17    0.03    


In [34]:
cv_std['test_rmse'].mean()

0.9360840292969925

ALS

In [35]:
bsl_option = {
    'method' : 'als'
    }

algo = BaselineOnly(bsl_options = bsl_option)

cv_std = cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


In [36]:
cv_std['test_rmse'].mean()

0.943963283996824

## HYPERPARAMETER TUNING

In [40]:
param_grid = {
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.4, 0.5]
    }

gs = GridSearchCV(SVD, param_grid, measures=['RMSE', 'MAE'], cv = 3)
gs.fit(data)

In [41]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.9449970381962047
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


In [42]:
print(gs.best_score['mae'])
print(gs.best_params['mae'])

0.7459244670707634
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
